In [1]:
import pymysql
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# DB 연결 - db 정보 업데이트 필요
connection = pymysql.connect(host='13.125.49.128', port=52795, user='root', password='asd456',
                       db='chatbotBase', charset='utf8')

# Now, we'll create a cursor and use it to execute a SELECT query
with connection.cursor() as cursor:
    sql = "SELECT uuid, course_data FROM chat_bot2"
    cursor.execute(sql)
    
    # Fetch all the rows from the query
    rows = cursor.fetchall()
    
    # Create a Pandas DataFrame from the rows
    df = pd.DataFrame(rows)
    
    # You can use the DataFrame's `head` method to preview the first few rows of the data
    print(df.head())

# Don't forget to close the connection when you're done
connection.close()

                                                   0  \
0  001354e236075d013c8eb4b53120ab3571152522e28544...   
1  001e386060c70832a656deeed81d014a983ef2e0577618...   
2  00488bc91a30477b87c75d3c7289f8c43c59b3454f1935...   
3  004f1f6318d64de636b7160f8725d0482c3ddd6db845c5...   
4  006b84448a329c034747dfe4708897ddb8970e114155dd...   

                              1  
0                     솔리디티 깨부수기  
1  디자이너의 스킬업을 위한 Lottie 완벽 이해하기  
2     게임 개발을 위해 알아야 할 Azure CDN  
3             Python 입문자를 위한 강의  
4            따라하면 취업되는 게임기획 강의!  


['uuid', 'url', 'course_data', 'categorie_data', 'keyword_data', 'course_info_data', 'grade_data', 'stu_num_data', 'teacher_data', 'playtime_data', 'level_data', 'price_data', 'review_data', 'dibs_num_data', 'teacher_info_data', 'course', 'categorie', 'keyword', 'course_info', 'grade', 'stu_num', 'teacher', 'playtime', 'level', 'price', 'review', 'dibs_num', 'teacher_info', 'ori_data']

In [2]:
df2 = df.copy()

In [3]:
# 데이터프레임 컬럼명 변경
df2.columns = ['uuid', 'course_data']

In [4]:
df2['course_data']

0                                            솔리디티 깨부수기
1                         디자이너의 스킬업을 위한 Lottie 완벽 이해하기
2                            게임 개발을 위해 알아야 할 Azure CDN
3                                    Python 입문자를 위한 강의
4                                   따라하면 취업되는 게임기획 강의!
                             ...                      
1563                              초보자를 위한 R프로그래밍 핵심 교재
1564       스프링 배치 - Spring Boot 기반으로 개발하는 Spring Batch
1565    [김왼손의 왼손코딩] 미운코딩새끼: 4시간 만에 끝내는 파이썬 기초 (전자책 포함)
1566          개발자는 왜 간단한 것도 안 된다고 하나요? (개발자와 커뮤니케이션하기)
1567                    머신러닝/딥러닝 소개 및 학습을 위한 파이썬 속성 과정
Name: course_data, Length: 1568, dtype: object

In [5]:
# Now, we'll use the TfidfVectorizer to convert the documents into numerical vectors
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df2['course_data'])

# We'll create a new column in the DataFrame to store the vectors
df2['vectors'] = list(vectors.toarray())

# Next, we'll define a function to find the top 5 most similar documents to a given input sentence
def get_similar_documents(df, input_sentence):
    # Convert the input sentence into a vector
    input_vector = vectorizer.transform([input_sentence])

    # Calculate the similarity between the input vector and the vectors in the DataFrame
    dit = dict()
    for idx, x in enumerate(cosine_similarity(input_vector.toarray(), df['vectors'].tolist())[0]):
        dit[idx] = x 
        
    # Return the top 5 most similar documents
    return df.iloc[[x[0] for x in sorted(dit.items(), key=lambda x : x[1])[-5:]]]['course_data']

# Let's test the function with a sample input sentence
input_sentence = "파이썬 초급 강의"
print(get_similar_documents(df2, input_sentence))

917       파이썬 무료 강의 (활용편7) - 머신러닝
1114             Azure 가상컴퓨터 - 초급
1094         코딩 초보자를 위한 파이썬 입문 강의
1214            입문자를 위한 파이썬 기초 강의
1533    SwiftUI 초급 강의 - 기본 개념 익히기
Name: course_data, dtype: object


In [6]:
input_sentence = "Azure"
print(get_similar_documents(df2, input_sentence))

1373    AZ-900 Microsoft Azure Fundamentals 자격증 준비하기
2                          게임 개발을 위해 알아야 할 Azure CDN
549                        게임 개발을 위해 알아야 할 Azure CDN
814                                Azure 기초 (AZ-900)
1114                                Azure 가상컴퓨터 - 초급
Name: course_data, dtype: object


In [ ]:
df